In [2]:
import sys
sys.path.append('..')

In [3]:
import numpy as np
import h5py
import os
from os.path import join
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
import yaml
import torch
# import ipyplot
import matplotlib
import cv2
import glob
import itertools
from tqdm.notebook import tqdm

from dataset import data_utils
from utils.train_utils import get_dynamics_dataset, get_readout_dataset, get_readout_seq_paths
from utils.cpconfig import update_to_ccn2_paths
from utils.viz_utils import compose_pred, gridimg, imshowreal, plot_sequence_images, save_video
from dataset.data_utils import SCENARIOS

In [4]:
train_paths = get_readout_seq_paths("/ccn2/u/rmvenkat/chpatel/physion_dataset/", "Dominoes", 'train', 0.9)
val_paths = get_readout_seq_paths("/ccn2/u/rmvenkat/chpatel/physion_dataset/", "Dominoes", 'val', 0.9)


pilot_dominoes_4mid_boxroom_2_0008.hdf5
pilot_dominoes_1mid_J025R45_boxroom_0068.hdf5
pilot_dominoes_SJ020_d3chairs_o1plants_tdwroom_0055.hdf5
pilot_dominoes_2mid_J020R15_d3chairs_o1plants_tdwroom_0048.hdf5
pilot_dominoes_2mid_J020R15_d3chairs_o1plants_tdwroom_2_0091.hdf5
pilot_dominoes_2mid_J025R30_tdwroom_0033.hdf5
pilot_dominoes_4mid_boxroom_2_0014.hdf5
pilot_dominoes_1mid_J025R45_boxroom_0022.hdf5
pilot_dominoes_default_boxroom_0051.hdf5
pilot_dominoes_2mid_J020R15_d3chairs_o1plants_tdwroom_0003.hdf5
pilot_dominoes_4mid_boxroom_2_0022.hdf5
pilot_dominoes_SJ020_d3chairs_o1plants_tdwroom_0087.hdf5
pilot_dominoes_2mid_J025R30_tdwroom_0075.hdf5
pilot_dominoes_2mid_J020R15_d3chairs_o1plants_tdwroom_2_0077.hdf5
pilot_dominoes_2mid_J020R15_d3chairs_o1plants_tdwroom_0080.hdf5
pilot_dominoes_4midRM1_tdwroom_0009.hdf5
pilot_dominoes_0mid_d3chairs_o1plants_tdwroom_0001.hdf5
pilot_dominoes_2mid_J020R15_d3chairs_o1plants_tdwroom_2_0068.hdf5
pilot_dominoes_1mid_J025R45_boxroom_0013.hdf5
pilot_do

In [ ]:
cfg = yaml.safe_load("""
data:
  dataset_class: physion
  physion_path: /home/chpatel_stanford_edu/physion_dataset
  saved_data_path: /home/chpatel_stanford_edu/physion_dataset/saved_data
  protocol : all
  seq_len : -1  # for whole sequence
  skip: 3
  background: False
  depth: False
""")
cfg = update_to_ccn2_paths(cfg)

In [ ]:
dataset = get_dynamics_dataset(cfg['data'])

In [ ]:
dataloader_iter = iter(dataloader)

In [ ]:
sample = next(dataloader_iter)

In [ ]:
print(sample['depth'].min(), sample['depth'].max(), sample['depth'].mean(), sample['depth'].median(), sample['depth'].std())
a = sample['depth'].log()
print(a.min(), a.max(), a.mean(), a.median(), a.std())

In [ ]:
inp_dict = prep_inputs(sample, torch.device('cpu'))

In [ ]:
print(sample.keys())
print(sample['rgb_unnorm'].shape)
print(sample['seg'].shape)
print(inp_dict.keys())
print(inp_dict['imgs'].shape)

In [ ]:
plt.imshow(inp_dict['gt'][0].permute(1,2,0).numpy()); plt.axis('off'); plt.show()
plt.imshow(inp_dict['gt_mask'][0].permute(1,2,0).numpy()); plt.axis('off'); plt.show()
plt.imshow(sample['seg'][0, 0].numpy()); plt.axis('off'); plt.show()
# plt.imshow(inp_dict['imgs'][0, 0].permute(1,2,0).numpy()); plt.axis('off'); plt.show()
plt.imshow(sample['rgb_unnorm'][0, 0].numpy()); plt.axis('off'); plt.show()
plt.imshow(sample['depth'][0, 0, 0].numpy()); plt.axis('off'); plt.show()
plt.imshow(sample['depth'][0, 0, 0].clamp(0., 6.).numpy()); plt.axis('off'); plt.show()


In [ ]:
np.log(8)

In [ ]:
# plt.imshow(inp_dict['gt'][0].permute(1,2,0).numpy()); plt.axis('off'); plt.show()
# pts = (inp_dict['sample_pts'].numpy() * 255).astype(int)[0]
# sam_msk = (inp_dict['sample_mask'].numpy() ).astype(int)[0]
# sam_rgb = (inp_dict['sample_rgb'].numpy() ).astype(int)[0]

# msk = np.zeros([256,256,3], dtype=int)
# msk[pts[:, 0], pts[:, 1], 0] = sam_msk[:, 0]
# msk[pts[:, 0], pts[:, 1], 1] = 1-sam_msk[:, 0]
# msk = msk*255
# plt.imshow(msk); plt.axis('off'); plt.show()

# rgb = np.ones([256,256,3], dtype=int) * 128
# rgb[pts[:, 0], pts[:, 1]] = sam_rgb
# plt.imshow(rgb); plt.axis('off'); plt.show()


In [ ]:
pts.shape

In [ ]:
pth = '/home/chpatel_stanford_edu/physion_dataset/dynamics_training_depth/Dominoes/pilot_dominoes_4mid_boxroom/0005.hdf5'
# pth = '/home/chpatel_stanford_edu/physion_dataset/model_testing/Dominoes/pilot_dominoes_0mid_d3chairs_o1plants_tdwroom-redyellow_0001.hdf5'
with h5py.File(pth, 'r') as h5:
    print(h5['static'].keys())
    num_frames = data_utils.get_num_frames(h5)
    print(h5['frames']['0000']['images'].keys())
#     print(h5['frames']['0000']['images']['_depth_cam0'])
#     print(h5['frames']['0000']['images']['_img_cam0'])
#     print(h5['frames']['0000']['images']['_id_cam0'])
    imgs, segs, depths = data_utils.index_imgs(h5, np.arange(num_frames), get_depth=True)


In [ ]:
plot_sequence_images(np.clip(depths, 0, 4)/4)
# plot_sequence_images(depths)


In [ ]:
plot_sequence_images(imgs)


In [ ]:
print(depths.min(), depths.max())

In [ ]:
def save_physion_videos(scen):

    data_dir = "/ccn2/u/rmvenkat/chpatel/physion_dataset/dynamics_training/"
    out_dir = "/ccn2/u/rmvenkat/chpatel/data_viz/dynamics_training/" + scen
    if os.path.isdir(out_dir):
        print(f"{out_dir} already exists.")
        return
    os.mkdir(out_dir)

    all_files = sorted(glob.glob(join(data_dir, scen, '*.hdf5')))
    all_files = all_files[::5]
    for i, pth in tqdm(enumerate(all_files)):
        with h5py.File(pth, 'r') as h5:
            num_frames = data_utils.get_num_frames(h5)
            imgs, segs = data_utils.index_imgs(h5, np.arange(num_frames))
        fname = os.path.basename(pth).split('.')[0]
        save_video(imgs, join(out_dir, "{}.webm".format(fname)), 30)

# for scen in SCENARIOS:
for scen in ['Dominoes']:
    print(scen)
    save_physion_videos(scen)

Dominoes


0it [00:00, ?it/s]

OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
OpenCV: FFMPEG: tag 

In [ ]:
def get_depth_values(image: np.array, depth_pass: str = "_depth", width: int = 256, height: int = 256, near_plane: float = 0.1, far_plane: float = 100) -> np.array:
        """
        Get the depth values of each pixel in a _depth image pass.
        The far plane is hardcoded as 100. The near plane is hardcoded as 0.1.
        (This is due to how the depth shader is implemented.)
        :param image: The image pass as a numpy array.
        :param depth_pass: The type of depth pass. This determines how the values are decoded. Options: `"_depth"`, `"_depth_simple"`.
        :param width: The width of the screen in pixels. See output data `Images.get_width()`.
        :param height: The height of the screen in pixels. See output data `Images.get_height()`.
        :param near_plane: The near clipping plane. See command `set_camera_clipping_planes`. The default value in this function is the default value of the near clipping plane.
        :param far_plane: The far clipping plane. See command `set_camera_clipping_planes`. The default value in this function is the default value of the far clipping plane.
        :return An array of depth values.
        """

        # Convert the image to a 2D image array.
        image = np.flip(np.reshape(image, (height, width, 3)), 0)
        if depth_pass == "_depth":
            depth_values = np.array((image[:, :, 0] + image[:, :, 1] / 256.0 + image[:, :, 2] / (256.0 ** 2)))
        elif depth_pass == "_depth_simple":
            depth_values = image[:, :, 0] / 256.0
        else:
            raise Exception(f"Invalid depth pass: {depth_pass}")
        # Un-normalize the depth values.
        return (depth_values * ((far_plane - near_plane) / 256.0)).astype(np.float32)

In [ ]:
dimg = plt.imread('/home/chpatel_stanford_edu/physion_dataset/dominoes_with_depth/train/pilot_dominoes_0mid_d3chairs_o1plants_tdwroom/0000_map.png')

In [ ]:
plt.imshow(dimg)